In [1]:
!pip install -qU pymupdf4llm transformers sentence-transformers datasets bitsandbytes
!pip install orjson==3.9.15
!pip install -qU langchain chromadb langchain-chroma langchain_community faiss-gpu
!pip install langchain-huggingface
!pip install rank_bm25
!pip install peft

In [2]:
import pymupdf4llm
import re
from tqdm.notebook import tqdm
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.schema.runnable import RunnablePassthrough
from langchain.retrievers import BM25Retriever, EnsembleRetriever, ContextualCompressionRetriever
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from langchain_huggingface import HuggingFacePipeline
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
import torch
import os
import pandas as pd
import pandas as pd
import numpy as np
import unicodedata

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def process_pdf(file_path, chunk_size=512, chunk_overlap=32):
    # PDF 파일 열기
    pdf = pymupdf4llm.to_markdown(file_path)
    pdf =  re.sub(r'(?<!\n)\n\n(?!\n)', '\n', pdf)
    doc = [Document(page_content=pdf, metadata={"source": file_path})]
    # pdf를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_documents(doc)
    # Document 객체 리스트 생성
    return chunk_temp

def create_vector_db(document, embeddings, file_path):
    db = Chroma.from_documents(
    documents=document,
    embedding=embeddings, persist_directory=file_path)
    return db

def process_pdfs_from_dataframe(base_directory, embeddings, chunk_size, chunk_overlap):
    """딕셔너리에 pdf명을 키로 해서 DB, retriever 저장"""
    pdf_databases = {}
    for p in tqdm(os.listdir(base_directory), desc="Creating Vectordb for each PDF"):
        print(p)
        doc = process_pdf(base_directory + p, chunk_size, chunk_overlap)
        print(doc)

        # base_directory에서 train_source 또는 test_source 디렉토리를 찾아서 파일 경로 결정
        if base_directory.split('/')[-2] == 'train_source':
            file_path = base_directory[:base_directory.find('train_source/')]
        elif base_directory.split('/')[-2] == 'test_source':
            file_path = base_directory[:base_directory.find('test_source/')]

        # vectordb의 저장 경로 설정
        vectordb_path = file_path + 'pymupdf4llm/' + p[:p.find('.pdf')]

        # 경로에 파일이 이미 존재하는지 확인
        if os.path.exists(vectordb_path):
            print(f"Vector DB already exists at {vectordb_path}, loading existing DB...")
            vectordb = Chroma(persist_directory=vectordb_path, embedding_function=embeddings)
        else:
            print(f"Creating new Vector DB at {vectordb_path}...")
            vectordb = create_vector_db(doc, embeddings, vectordb_path)

        # retriever = vectordb.as_retriever(search_kwargs={"k": 3})
        pdf_databases[unicodedata.normalize('NFC', p.split('.pdf')[0])] = {
            'db': vectordb,
            # 'retriever': retriever,
            'doc': doc
        }

    return pdf_databases

## Multiple Vector for each pdf

In [5]:
model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
bge_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
base_directory = '/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/test_source/'
pdf_databases = process_pdfs_from_dataframe(base_directory, bge_embeddings, chunk_size=1600, chunk_overlap=50)

Creating Vectordb for each PDF:   0%|          | 0/9 [00:00<?, ?it/s]

「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.pdf
[Document(metadata={'source': '/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.pdf'}, page_content='#### FIS\n# ISSUE & FOCUS\n\n\n**제1호**\n**2022.03.**\n\n\n**발행인 박용주   발행처 04637 서울특별시 중구 퇴계로 10(남대문로5가 537) 메트로타워   T 02)6908-8568   F 02)6312-8959** www.kpfis.or.kr\n**작성 심혜인 부연구위원, 정성호 연구위원  자료수집 최성진 사원**\n\n\n##### 우발부채\n**Contingent Liabilities**\n**1** 들어가며\n**2** 우발부채 개념 및 용어 정의\n**3** 국가결산보고서 우발부채 현황\n**4** 국제기준(GFS)에 근거한 우발부채 분류 재검토\n**5** 나가며\n\n\n-----\n**FIS** **ISSUE & FOCUS**\n최근 국제기준의 재정통계 적용과 관련해 우발부채에 관한 구체적 논의가 진행 중이다. 우발부\n채의 관리는 지속가능한 재정운영 차원에서 중요한 이슈인데, 이는 미래의 다양한 의제의무를\n### 우발부채\n포함하고 있기 때문이다.\n**(Contingent Liabilities)**\n우리나라는 국가결산보고서 등에서 우발부채를 인식하고 있으나 용어 사용에 혼란이 있고 분\n류에 대한 개념적 논거가 명확하지 않은 면이 있다. 이 글에서는 국제기준에 근거하여 우발부채의 개념에 대한 일관된 인식 프레임을\n제시하고, 국가결산 정보를 활용한 심층분석을 통하여 우리나라 우발부채 현황을 체계적으로 파악해보고자 하였다. 이를 통해 독자가\n우발부채 관리의 중요성에 대해

In [7]:
pdf_databases

{'「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》': {'db': <langchain_chroma.vectorstores.Chroma at 0x78d26f903bb0>,
  'doc': [Document(metadata={'source': '/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.pdf'}, page_content='#### FIS\n# ISSUE & FOCUS\n\n\n**제1호**\n**2022.03.**\n\n\n**발행인 박용주   발행처 04637 서울특별시 중구 퇴계로 10(남대문로5가 537) 메트로타워   T 02)6908-8568   F 02)6312-8959** www.kpfis.or.kr\n**작성 심혜인 부연구위원, 정성호 연구위원  자료수집 최성진 사원**\n\n\n##### 우발부채\n**Contingent Liabilities**\n**1** 들어가며\n**2** 우발부채 개념 및 용어 정의\n**3** 국가결산보고서 우발부채 현황\n**4** 국제기준(GFS)에 근거한 우발부채 분류 재검토\n**5** 나가며\n\n\n-----\n**FIS** **ISSUE & FOCUS**\n최근 국제기준의 재정통계 적용과 관련해 우발부채에 관한 구체적 논의가 진행 중이다. 우발부\n채의 관리는 지속가능한 재정운영 차원에서 중요한 이슈인데, 이는 미래의 다양한 의제의무를\n### 우발부채\n포함하고 있기 때문이다.\n**(Contingent Liabilities)**\n우리나라는 국가결산보고서 등에서 우발부채를 인식하고 있으나 용어 사용에 혼란이 있고 분\n류에 대한 개념적 논거가 명확하지 않은 면이 있다. 이 글에서는 국제기준에 근거하여 우발부채의 개념에 대한 일관된 인식 프레임을\n제시하고, 국가결산 정보를 활용한 심층분석을 통하여 우리나라

In [8]:
test_df = pd.read_csv('/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/test.csv')
test_df

,SAMPLE_ID,Source,Source_path,Question
0,TEST_000,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
1,TEST_001,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?
2,TEST_002,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?
3,TEST_003,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?
4,TEST_004,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요?
...,...,...,...,...
93,TEST_093,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,재정정책에서 공적보증채무와 다른 일회성 보증은 어떻게 구분되는가?
94,TEST_094,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,미래사회보장급여에 대한 순의무란 무엇을 의미하는가?
95,TEST_095,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 어떻게...
96,TEST_096,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,"우발부채란 무엇이며, 그 관리가 왜 중요한가?"


In [9]:
from transformers import Gemma2ForCausalLM
import torch

def setup_llm_pipeline():
    # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # 모델 ID
    model_id = "rtzr/ko-gemma-2-9b-it"

    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = Gemma2ForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True )

    # adapter_name = "/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/pymupdf4llm/q&a_finetune/checkpoint-300"
    # model = PeftModel.from_pretrained(model, adapter_name,is_trainable=False)

    # adapter_name = "/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/pymupdf4llm/prompt_finetune/checkpoint-500"
    # model = PeftModel.from_pretrained(model, adapter_name,is_trainable=False)

    adapter_name = "/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/pymupdf4llm/q&a_finetune_cleaned/checkpoint-600"
    model = PeftModel.from_pretrained(model, adapter_name,is_trainable=False)

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        task="text-generation",
        return_full_text=False,
        max_new_tokens=512,
        repetition_penalty=1.0,
        do_sample=False,
        # temperature = 0.1
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return hf

hf = setup_llm_pipeline()

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

In [10]:
reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

In [11]:
def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)

sub_list = []
for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Answering Questions"):
    retriever = pdf_databases[unicodedata.normalize('NFC', row['Source'])]['db'].as_retriever(search_kwargs={"k":5})
 
    # 상위 2개의 문서 선택
    compressor = CrossEncoderReranker(model=reranker_model, top_n=2)

    # 문서 압축 검색기 초기화
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor, base_retriever=retriever
    )

    template = """
    다음 정보를 바탕으로 질문에 답하세요:
    {context}

    ### 질문: {question}

    질문의 핵심만 파악하여 간결하게 1-2문장으로 답변하고, 불필요한 설명은 피하며 요구된 정보만 제공하세요.

    ### 답변:
    """
    prompt = ChatPromptTemplate.from_template(template)

    rag_chain = (
        {
            "context": compression_retriever | format_docs,
            "question": RunnablePassthrough(),
        }
        | prompt
        | hf
        | StrOutputParser()
    )

    question = row['Question']
    print('------------------------')
    print('질문: ', question)
    print('------------------------')
    answer = rag_chain.invoke(question).strip()
    print('답변: ', answer)

    sub_list.append(answer)

Answering Questions:   0%|          | 0/98 [00:00<?, ?it/s]

------------------------
질문:  2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
------------------------


답변:  2,300,000백만원
------------------------
질문:  중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?
------------------------


답변:  ㅇ (창업기반지원) 기술력과 사업성이 우수하고 미래 성장가능성이 높은 중소벤처기업의 창업을 활성화하고 고용창출 도모
    ㅇ (개발기술사업화) 중소기업이 보유한 우수 기술의 사장을 방지하고 개발기술의 제품화·사업화를 촉진하여 기술기반 중소기업을 육성
------------------------
질문:  중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?
------------------------


답변:  중소기업진흥에 관한 법률 제66조, 제67조, 제74조
------------------------
질문:  2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?
------------------------


...

답변:  취약계층의 에너지 비용 부담 완화를 위해 지속적 지원단가 현실화 추진
------------------------
질문:  행복주택출자 사업은 어떤 근거로 추진되고 있는가?
------------------------


답변:  행복주택출자 사업은 주택도시기금법 제9조제1항가목 및 공공주택특별법 제2조1호가목에 따라 추진되고 있다.
------------------------
질문:  행복주택출자 사업은 어떤 목적으로 시행되는가?
------------------------


답변:  행복주택출자 사업은 국민의 행복주거 실현을 위한 보편적 주거복지 정책의 일환으로 도심 내 다양한 부지를 활용하여 행복주택을 공급하기 위해 시행된다.
------------------------
질문:  행복주택출자 사업의 주요 수혜자는 누구인가?
------------------------


답변:  대학생・사회초년생・신혼부부 등 젊은층(80%), 고령자 및 주거취약계층(20%)
------------------------
질문:  행복주택출자 사업의 사업비 추이는 어떠한가?
------------------------


답변:  2022년 564,342백만원에서 2023년 684,607백만원으로 증가하고, 2024년 532,913백만원으로 감소하는 추세이다.
------------------------
질문:  행복주택출자 사업의 사업시행주체는 누구인가?
------------------------
답변:  한국토지주택공사(LH), 지자체(지방공사)
------------------------
질문:  국고보조사업의 보조율은 어떠한 기준에 따라 운용되는가?
------------------------
답변:  보조금법에 따라 기준보조율과 차등보조율을 적용하여 운용하고 있다.
------------------------
질문:  프랑스의 재정조정제도에서 최근 강조되는 형평교부금은 어떤 역할을 하는가?
------------------------
답변:  최근 중앙정부의 재정적자로 인한 수평적 형평교부금은 재정불균형을 해소하는 역할을 한다.
------------------------
질문:  지방재정조정제도의 목적은 무엇인가?
------------------------
답변:  지방재정조정제도의 목적은 지방자치단체의 부족한 재원을 보충하여 중앙-지방 간 수직적 불균형 완화 또는 재정력 격차 완화를 통해 국가 재정의 재배분 목적을 충족하여 지방자치단체의 기본적인 수요에 대한 지출을 촉진하는 것이다.
------------------------
질문:  국제적으로 성과중심 재정관리 강화 움직임이 확산된 시기는 언제인가?
------------------------
답변:  2000년대 후반 금융위기 이후
------------------------
질문:  한국의 재정사업 성과관리제도는 어떠한 법을 통해 운영되고 있으며, 성과관리 기본계획과 추진계획은 어떻게 의무화되었는가?
------------------------
답변:  한국 재정사업 성과관리제도는 「국가재정법」 제8조(성과중심의 재정운용)를 통해 운영되며, 성과관리 기본계획은 5개년 단위로, 성과관리 추진계획은 연간 단위

In [12]:
submission_df = pd.read_csv('/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/sample_submission.csv')
submission_df

,SAMPLE_ID,Answer
0,TEST_000,데이콘
1,TEST_001,데이콘
2,TEST_002,데이콘
3,TEST_003,데이콘
4,TEST_004,데이콘
...,...,...
93,TEST_093,데이콘
94,TEST_094,데이콘
95,TEST_095,데이콘
96,TEST_096,데이콘


In [13]:
submission_df.loc[:, 'Answer'] = np.array(sub_list)

In [14]:
submission_df.to_csv('/content/drive/MyDrive/재정정보 AI 검색 알고리즘 경진대회/pymupdf4llm/pymupdf4llm_multiple_vectordb_q&a_finetune_cleaned_checkpoint_600_top_n_2.csv', index=False)